In [ ]:
import os
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from openpyxl import load_workbook

from scipy import stats as stats
from copy import copy as copy



In [ ]:
# from functions.functions import test_print

# from functions.plotting import plot_SA_Hist
# from functions.plotting import draw_probe_plot
# from functions.plotting import probe_GeoMean_Plots
# from functions.plotting import binding_density_plot

from functions.masterdata import master_data



In [ ]:
# read in paths from config file

configDict = {
    'rootDir': '',
    'initialDataPath' : '',
    'QCDataPath' : '',
    # 'labWorksheet01Path':'',
    'projectName':''
}

# with open(os.path.join(workingDir, 'config.txt'),'r') as f:
with open('config.txt','r') as f:
    lines = f.readlines()
    for line in lines:
        if not line.startswith('#'):
            line = line.strip()
            fields = line.split(':')

            if fields[0].strip()=='initialDataPath':
                configDict[fields[0].strip()] = fields[1].strip().strip('\'')
            else:
                configDict[fields[0].strip()] = fields[1].strip().strip('\'')
## ToDo: Add checks to ensure that minimal fields have been populated. Raise errors or warnings


In [ ]:
configDict

# Define functions

In [ ]:
class threshold_probes:
    def __init__(self, data, bins):
        
        self.data = data.drop(labels=['mean','probeClass'], axis=1)
        self.bins = bins
        self.thisHist = plt.hist(self.data.values.flatten(), bins = self.bins)

    def zoom_plot(self, start, end):
        plt.hist(self.data.values.flatten(), bins = self.bins)
        plt.xlim(0,3)        
        
    def check_threshold(self, start, end):
        print(self.thisHist[0][start:end])
        print(self.thisHist[1][start:end])

    def set_threshold_idx(self, idx):
        print(self.thisHist[0][idx])
        print(self.thisHist[1][idx])
        
        self.threshold_idx = idx
        self.threshold = self.thisHist[1][idx]

    def get_filter(self):
        self.ETfilter = self.data >= self.threshold
        return(self.ETfilter)


# Import data from Nanostring initial dataset file.

In [ ]:
dataPath = os.path.join(configDict['rootDir'],configDict['initialDataPath'])
dataPath = os.path.join(configDict['rootDir'],configDict['QCDataPath'])

masterData = master_data(dataPath)


counts, probes, segs = masterData.get_data()


counts

# probes

# segs

ToDo: Need to check that column names are unique in SegmentProperties!! KO has 2 entries in current version, need to trace down where this has been introduced.

In [ ]:
dropCols = ['SlideName', 'ScanLabel', 'ROILabel', 'SegmentLabel',
            'SegmentDisplayName', 'Origin Instrument ID', 'AOISurfaceArea', 
            'AOINucleiCount', 'ROICoordinateX', 'ROICoordinateY', 
            'RawReads', 'AlignedReads', 'DeduplicatedReads', 'TrimmedReads', 
            'StitchedReads', 'SequencingSaturation', 'SequencingSetID', 
            'UMIQ30', 'RTSQ30', 'GeoMxNgsPipelineVersion', 'LOT_Mouse_NGS_Whole_Transcriptome_Atlas_RNA_1_0',
            'ROIID', 'SegmentID', 'ScanWidth', 'ScanHeight', 'ScanOffsetX', 'ScanOffsetY']

keepCols = [x for x in segs.columns if not x in dropCols]

for c in keepCols:
    print(c)
    print(len(set(segs[c])))
    print(set(segs[c]))
    print()

ToDo: Save drop cols to config file.

   - Work out workflow to allow importing previous drop cols, and working with remaining drop cols to get final list (saving final dropped cols back to config file)





In [ ]:
sampleInfoExternal = masterData.segData.copy()

In [ ]:
sampleInfoExternal.columns

In [ ]:
sampleInfoExternal['TrimmedReads']

In [ ]:
print(masterData.probeClass)
print(masterData.probeClassDict)
# dataLog1External, sampleInfoExternal = masterData.drop_AOIs('#16_9', writeOrig=True)

nuclei = sampleInfoExternal['AOINucleiCount']
# print(nuclei)
surfArea = sampleInfoExternal['AOISurfaceArea']
# print(surfArea)

# dataLog1External, sampleInfoExternal = masterData.add_class_mean(masterData.dataLog1)

In [ ]:
# os.getcwd()

In [ ]:
# configDict

In [ ]:
# dataPath = os.path.join(configDict['rootDir'],configDict['initialDataPath'])

# masterData = master_data(dataPath)
# dataLog1External, sampleInfoExternal = masterData.get_data()

# # For the Thomas TMA study there are a number of additional AOIs to be used for data normalisation studies.
# # These should be removed before further data analysis and QC.

# print(masterData.probeClass)
# print(masterData.probeClassDict)
# # dataLog1External, sampleInfoExternal = masterData.drop_AOIs('#16_9', writeOrig=True)

# nuclei = sampleInfoExternal.loc['AOI nuclei count']
# surfArea = sampleInfoExternal.loc['AOI surface area']

# dataLog1External, sampleInfoExternal = masterData.add_class_mean(masterData.dataLog1)

In [ ]:
# ### import data from excel workbook

# # wb = load_workbook('../../../Documents/Nanostring/projects/Dando/DSP_Analysis_Suite/Initial Dataset.xlsx')
# wb = load_workbook('../../../Documents/Nanostring/projects/McGovern/msWTA/Initial Dataset.xlsx')

# print(wb.sheetnames)


In [ ]:
# ws = wb['SegmentProperties']

# segments = [[y.value for y in x] for x in ws[ws.calculate_dimension()]]
# segments

In [ ]:
# ### Convert nested list to a pandas dataFrame and extract expression data with labels
# segs = pd.DataFrame(segments)
# segs.rename(columns=segs.loc[0], inplace=True)
# segs.drop(index=0, inplace=True)
# segs.reset_index(inplace=True, drop=True)
# segs

In [ ]:
# segs.loc[segs['SlideName'] == 'Ms_OB_TMEM_8um_2022_08_18'].shape

In [ ]:
slides = set(segs['SlideName'])

colLookup = dict(zip(slides,range(len(slides))))

In [ ]:
colLookup

In [ ]:
colMap = [colLookup[x]/len(slides) for x in segs['SlideName']]

# colMap = [colLookup[x] for x in segs['SlideName']]


In [ ]:
colMap 

In [ ]:
sum(segs['RawReads'])

In [ ]:
# plot aoi size vs read number

ToDo: Convert the following to subplots in a single figure

 - Add titles
 - Add legends

In [ ]:
fig, ax = plt.subplots(5,2, figsize=[20,20])


ax[0][0].scatter(segs['AOISurfaceArea'],segs['RawReads'],alpha=0.5, c=colMap, cmap='nipy_spectral')
ax[0][0].set_title('AOISurfaceArea vs Raw reads')
ax[0][0].set_xlabel('Surface Area')
ax[0][0].set_ylabel('Raw Reads')

ax[1][0].scatter(segs['AOISurfaceArea'],(segs['DeduplicatedReads']/segs['AOISurfaceArea']),alpha=0.5, c=colMap, cmap='nipy_spectral')
ax[1][0].set_title('AOISurfaceArea vs DeduplicatedReads/AOISurfaceArea')
ax[1][0].set_xlabel('Surface Area')
ax[1][0].set_ylabel('DeduplicatedReads/AOISurfaceArea')

ax[3][0].scatter(segs['AOISurfaceArea'],segs['DeduplicatedReads'],alpha=0.5, c=colMap, cmap='nipy_spectral')
ax[3][0].set_title('AOISurfaceArea vs DeduplicatedReads')
ax[3][0].set_xlabel('Surface Area')
ax[3][0].set_ylabel('DeduplicatedReads')

ax[4][0].scatter(segs['AOISurfaceArea'],segs['SequencingSaturation'],alpha=0.5, c=colMap, cmap='nipy_spectral')
ax[4][0].set_title('AOISurfaceArea vs SequencingSaturation')
ax[4][0].set_xlabel('Surface Area')
ax[4][0].set_ylabel('SequencingSaturation')



ax[0][1].scatter(segs['RawReads'],segs['DeduplicatedReads'],alpha=0.5, c=colMap, cmap='nipy_spectral')
ax[0][1].semilogx()
ax[0][1].semilogy()
ax[0][1].set_title('RawReads vs DeduplicatedReads')
ax[0][1].set_xlabel('RawReads')
ax[0][1].set_ylabel('DeduplicatedReads')

ax[3][1].scatter(segs['SequencingSaturation'],segs['DeduplicatedReads']/segs['AOISurfaceArea'],alpha=0.5, c=colMap, cmap='nipy_spectral')
ax[3][1].set_title('SequencingSaturation vs DeduplicatedReads/AOISurfaceArea')
ax[3][1].set_xlabel('Sequencing Saturation')
ax[3][1].set_ylabel('DeduplicatedReads/AOISurfaceArea')

ax[2][1].scatter(segs['SequencingSaturation'],segs['RawReads']/segs['AOISurfaceArea'],alpha=0.5, c=colMap, cmap='nipy_spectral')
ax[2][1].set_title('SequencingSaturation vs Raw reads/AOISurfaceArea')
ax[2][1].set_xlabel('SequencingSaturation')
ax[2][1].set_ylabel('Raw Reads/AOISurfaceArea')

ax[1][1].scatter(segs['RawReads'],segs['SequencingSaturation'],alpha=0.5, c=colMap, cmap='nipy_spectral')
ax[1][1].semilogx()
ax[1][1].set_title('RawReads vs SequencingSaturation')
ax[1][1].set_xlabel('RawReads')
ax[1][1].set_ylabel('SequencingSaturation')

ax[4][1].scatter(segs['DeduplicatedReads'],segs['SequencingSaturation'],alpha=0.5, c=colMap, cmap='nipy_spectral')
ax[4][1].semilogx()
ax[4][1].set_title('DeduplicatedReads vs SequencingSaturation')
ax[4][1].set_xlabel('DeduplicatedReads')
ax[4][1].set_ylabel('SequencingSaturation')


plt.tight_layout()
plt.savefig("QC_Plots.svg")
plt.show()

## Set up as selectable x and y axes using 

In [ ]:
# plt.scatter(segs['AOISurfaceArea'],segs['RawReads'],alpha=0.5, c=colMap, cmap='nipy_spectral')

In [ ]:
# plt.scatter(segs['SequencingSaturation'],segs['RawReads']/segs['AOISurfaceArea'],alpha=0.5, c=colMap, cmap='nipy_spectral')
# plt.semilogy()

In [ ]:
# plt.scatter(segs['SequencingSaturation'],segs['DeduplicatedReads']/segs['AOISurfaceArea'],alpha=0.5, c=colMap, cmap='nipy_spectral')


In [ ]:
# plt.scatter(segs['AOISurfaceArea'],segs['DeduplicatedReads'],alpha=0.5, c=colMap, cmap='nipy_spectral')
# plt.semilogy()

In [ ]:
# plt.scatter(segs['AOISurfaceArea'],(segs['DeduplicatedReads']/segs['AOISurfaceArea']),alpha=0.5, c=colMap, cmap='nipy_spectral')
# plt.semilogy()

In [ ]:
# plt.scatter(segs['RawReads'],segs['DeduplicatedReads'],alpha=0.5, c=colMap, cmap='nipy_spectral')
# plt.semilogx()
# plt.semilogy()

In [ ]:
# segs['SequencingSaturation'].mean()

In [ ]:
# segs['SequencingSaturation'].min()

In [ ]:
# segs['SequencingSaturation'].max()

In [ ]:
# sum(segs['SequencingSaturation']<50.0)

In [ ]:
# sum(segs['SequencingSaturation']>=50.0)

In [ ]:
# plt.scatter(segs['AOISurfaceArea'],segs['SequencingSaturation'],alpha=0.25, c=colMap, cmap='nipy_spectral')

In [ ]:
# plt.scatter(segs.loc[segs['SlideName'] == 'RB15P48811_SOETENS. 1.A']['AOISurfaceArea'],segs.loc[segs['SlideName'] == 'RB15P48811_SOETENS. 1.A']['SequencingSaturation'])

# # segs.loc[segs['SlideName'] == 'RB15P48811_SOETENS. 1.A']['AOISurfaceArea']

In [ ]:
# plt.scatter(segs['DeduplicatedReads'],segs['SequencingSaturation'],alpha=0.25, c=colMap, cmap='nipy_spectral')
# plt.semilogx()


In [ ]:
# plt.scatter(segs['RawReads'],segs['SequencingSaturation'],alpha=0.2, c=colMap, cmap='nipy_spectral')
# plt.semilogx()
# plt.xlim(0,1E8)
# plt.ylim(0,100)
# # 

In [ ]:
# plt.scatter(segs.loc[segs['SlideName'] == 'RB15P48811_SOETENS. 1.A']['RawReads'],segs.loc[segs['SlideName'] == 'RB15P48811_SOETENS. 1.A']['SequencingSaturation'])
# plt.xlim(0,1E8)
# plt.ylim(0,100)


In [ ]:
# plt.scatter(segs['RawReads'],segs['DeduplicatedReads'],alpha=0.25, c=colMap, cmap='nipy_spectral')
# plt.semilogx()
# plt.semilogy()


# Investigate NTC wells


ToDo: Load in DCC data and parse info

 - Tie in with config file

In [ ]:
dataPath = "/Users/upton6/Library/CloudStorage/OneDrive-QueenslandUniversityofTechnology/Documents/Nanostring/Projects/NS_msWTA/DND"

In [ ]:
files = os.listdir(dataPath)
files = [x for x in files if x.endswith('.dcc')]

In [ ]:
files

In [ ]:
thisFile = os.path.join(dataPath, "DSP-1001660011355-A-A01.dcc")
thisFile = os.path.join(dataPath, "DSP-1001660011354-B-A01.dcc")
# thisFile = os.path.join(dataPath, "DSP-1001660011355-A-A01.dcc")


In [ ]:
with open(thisFile, 'r') as f:
    lines = f.readlines()
    for line in lines:
        line = line.strip()
        print(line)
    # print(lines)

In [ ]:
# dccPath = '../../../Documents/Nanostring/projects/Dando/DCC-2023XXXX/'

In [ ]:
# walkOut = os.walk(dccPath)

In [ ]:
# files = list(walkOut)[0][2]

In [ ]:
# files

In [ ]:
# NTCs = []
# for f in files:
#     if f.endswith('A01.dcc'):
#         NTCs.append(os.path.join(dccPath,f))
# #         print(f)
    

In [ ]:
# NTCs

In [ ]:
# NTCs = []
# for ntc in files:
#     if ntc.endswith('A01.dcc'):
#         print(ntc)
# #         NTCs.append(os.path.join(dccPath,ntc))
#         with open (os.path.join(dccPath,ntc),'r') as f:
#             counts = []
#             lines = f.readlines()
#             for line in lines[25:-1]:
# #                 print(line)
#                 counts.append(line.strip().split(','))
#             ds = pd.Series([x[1] for x in counts], index = [x[0] for x in counts], name=ntc.split('-')[1])
#             print(ds.shape)
#             NTCs.append(ds)
# #         break
        
        
        
# #         print(f)
    

In [ ]:
# ds

In [ ]:
# NTCFrame = pd.concat(NTCs, axis=1)
# NTCFrame.fillna(0, inplace=True)

In [ ]:
# NTCFrame

In [ ]:
# NTCFrame = NTCFrame.astype('int32')

In [ ]:
# NTCFrame.dtypes

In [ ]:
# NTCFrame.sum()

In [ ]:

# for ntc in NTCs:
#     with open (ntc,'r') as f:
#         counts = []
#         lines = f.readlines()
#         for line in lines[25:-1]:
#             print(line)
#             counts.append(line.strip())
            
#         ds = pd.Series(counts, name=)
#     break
        
        

# Investigate Probe Matrix Counts


In [ ]:
# ws = wb['BioProbeCountMatrix']

# counts = [[y.value for y in x] for x in ws[ws.calculate_dimension()]]
# counts

In [ ]:
# ### Convert nested list to a pandas dataFrame and extract expression data with labels
# counts = pd.DataFrame(counts)
# counts.rename(columns=counts.loc[0], inplace=True)
# counts.drop(index=0, inplace=True)
# counts.reset_index(inplace=True, drop=True)
# counts

In [ ]:
counts.columns

In [ ]:
# list(counts.columns)[12:]

In [ ]:
# AOIs = list(counts.columns)[12:]

In [ ]:
# negOrdered = counts.loc[counts['CodeClass'] == 'Negative', AOIs].sum().sort_values().index

In [ ]:
# set(counts['CodeClass'])

In [ ]:
# plt.figure(figsize=(10,10))
# plt.plot(counts.loc[counts['CodeClass'] == 'Negative', AOIs][negOrdered].T)

In [ ]:
# There are no control probes in WTA

# plt.figure(figsize=(10,10))
# plt.plot(counts.loc[counts['CodeClass'] == 'Control', AOIs][negOrdered].T)

In [ ]:
# counts.index = counts['TargetName']

In [ ]:
# counts.drop(counts.columns[:12], axis=1, inplace = True)

In [ ]:
counts.values.flatten().min()

In [ ]:
counts = counts.astype(np.float32)
counts = np.log2(counts)
counts

In [ ]:
# plt.matshow(counts, cmap='coolwarm')

In [ ]:
geoCounts = counts.groupby(level=0).mean()
geoCounts

In [ ]:
# plt.matshow(geoCounts, cmap='coolwarm')

In [ ]:
break